In [ ]:
import os
import requests
import pandas as pd

In [187]:
# loading external dataset of airbnb listing 
df = pd.read_excel("Airbnb Listing Descriptions.xlsx", sheet_name="Sheet1")
df.dropna(inplace=True)
df.shape

In [3]:
from openai import OpenAI
client = OpenAI()

In [6]:
def call_chatgpt(prompt, model="gpt-4"):
    api_url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": "Bearer "+os.environ['OPENAI_API_KEY'],  # Replace YOUR_API_KEY with your actual OpenAI API key
        "Content-Type": "application/json"
    }
    data = {
        "model": model,  # or another model version like "gpt-4" if applicable
        "messages": [{"role": "user", "content": prompt}]
    }

    response = requests.post(api_url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()
    else:
        return response.text
    
def parse_content_response(response):
    return response['choices'][0]['message']['content']

def parse_response_into_pf(response):
    # Extracting the table from the response
    table_content = response['choices'][0]['message']['content']
    
    # Splitting into rows
    rows = table_content.split('\n')
    
    
    # Parsing rows
    data = []
    for row in rows:
        if row.strip():  # Checking if row is not just whitespace
            row = row.replace("*","")
            row = row.replace("\"","")
            columns = row.split('|')
            if len(columns) > 1:  # Making sure it's a valid row
                # Extracting and trimming whitespace from each column entry
                data.append([col.strip() for col in columns if col.strip()])
    
    
    result_df = pd.DataFrame(data)
    result_df.columns = result_df.iloc[0,:]
    result_df = result_df.iloc[1:,:]
    return result_df

In [254]:
# instructions = """please generate the table again,
# this time I'm giving you only the description, 
# please create a table with these columns: number of bedrooms, number of bathrooms, neighborhood, 
# shared (indicates if anything in the accommodation will be shared): """
instructions = """

I have a list of Airbnb property descriptions, 
and I need to extract and compile key information 
into a structured format suitable for analysis with pandas in Python. 
When the information is not present, it can be indicated "Not Specified". 
Could you provide (using your understanding and not regular expressions) 
the following attributes from each description 

0. Description (include the first 5 characters of the original description)
1. Number of Bedrooms
2. Type of Property (e.g., studio, apartment, loft, etc.)
3. Is any space shared? (return TRUE OR FALSE)
4. Overall vibes/atmosphere (return one of the following labels, see detail explanation below: MODERN, CHIC, ARTSY, COSMOPOLITAN, HISTORIC, COMFORTABLE)

CONTENT INSTRUCTIONS:

- Instructions for attribute 1: 
DO NOT USE value in column "name" to infer this. Only use value in column "description". 
When you can infer there's only 1 bed, for e.g., studio or loft, return 1.
Keep the output numerical only, e.g. 1

- Instructions for attribute 3:
If there's any indication the space is shared, e.g. explicit mention, or wording referring to the space as a "room"

- Instructions for attribute 4: 
Modern: Classify homes as modern if they feature contemporary design, sleek furnishings, and often incorporate new technologies and minimalist aesthetics.
Chic: Label as chic where the style is sophisticated, fashionable, and stylish, typically featuring trendy decor and a cosmopolitan vibe.
Artsy: Identify as artsy if the home includes unique artistic elements, vibrant colors, or displays of local artwork, creating a creative and inspirational environment.
Cosmopolitan: Mark as cosmopolitan if the space reflects a global sophistication, often with decor that includes international influences or a layout suited for a worldly traveler.
Historic: Classify as historic if the property retains architectural features from a specific era or is located in a notable NYC neighborhood with rich historical significance.
Comfortable: If features point to coziness, comfort. OR Everything that don't fit other categories. 

- Instructions for all columns: 
Please return the exact text in the way that it appears in the original description.

OUTPUT FORMATTING: table format, with header (the column attributes), followed by each row
with the information obtained.
To making parsing easy, let's agree to seperate each row by "\n", 
and 2) seperated with | for each attribute.
DO NOT include any explanation before or after
INCLUDE the original column name in the result

EXAMPLE FORMATTING
"Description|Number of Bedrooms|Type of Property|Special Features|Is any space shared?|Overall vibes/atmosphere"
"Easy access to all downtown|1|Apartment|Fast Wifi, Desk, Shared kitchen|TRUE|ARTSY"

"""

In [149]:
#gathering datasets 

indices = [[0, 10], [10, 20]]

full_df = pd.DataFrame()

for idx in indices: 
    print(f"Processing rows from index {idx[0]} to {idx[1]}")
    start = idx[0]
    end = idx[1]

    shortened_dataset = df.iloc[start:end,:]
    data_text = shortened_dataset.to_string(index=False)
    prompt = instructions + data_text
    print('Calling ChatGPT API, wait up!')

    try:
        response = call_chatgpt(prompt)
        tmp_df = parse_response_into_pf(response)
        tmp_df = pd.concat([shortened_dataset.reset_index(), tmp_df.reset_index()], axis=1)
        full_df = pd.concat([full_df, tmp_df], ignore_index=True, axis=0)
    except Exception as e:
        print(f"Failed to process data for indices {start} to {end}: {e}")

Processing rows from index 0 to 10
Calling ChatGPT API, wait up!
Processing rows from index 10 to 20
Calling ChatGPT API, wait up!
Failed to process data for indices 10 to 20: Reindexing only valid with uniquely valued Index objects


In [154]:
parse_response_into_pf(response)

,Description,Number of Bedrooms,Type of Property,Is anything space shared?,Overall vibes/atmosphere
1,"Private, quiet",1,Room,Not Specified,COMFORTABLE
2,Come stay in your,1,Apartment,TRUE,HISTORIC
3,With a Backyard! A,1,Apartment,Not Specified,COMFORTABLE
4,Ask about summer discounts!,1,Townhouse,Not Specified,ARTSY
5,The apartment is located,3,Duplex Apartment,Not Specified,COMFORTABLE
6,Seeking responsible and consider,1,Apartment,Not Specified,MODERN
7,PLEASE INQUIRE BEFORE BOOKING!,1,Studio,Not Specified,COMFORTABLE
8,"Sorry, this listing is",Not Specified,Not Specified,Not Specified,Not Specified
9,"Recently renovated Studio, Approximately",1,Studio,Not Specified,MODERN
10,Relaxing,Not Specified,Room,Not Specified,COMFORTABLE


In [167]:
# tmp_df = pd.concat([shortened_dataset.reset_index(), tmp_df.reset_index()], axis=1)
# pd.concat([full_df, tmp_df], ignore_index=True, axis=0)
tmp_df = parse_response_into_pf(response)
tmp_df = pd.concat([shortened_dataset.reset_index(), tmp_df.reset_index()], axis=1)
full_df.shape, tmp_df.shape

((10, 10), (10, 10))

In [170]:
full_df = pd.concat([full_df, tmp_df], ignore_index=True, axis=0)

In [171]:
#gathering datasets 

# indices = [[0, 10], [10,20], [20,30], [30,40]]
indices = [[20, 30], [30, 40]]

full_2_df = pd.DataFrame()

for idx in indices: 
    print(f"Processing rows from index {idx[0]} to {idx[1]}")
    start = idx[0]
    end = idx[1]

    shortened_dataset = df.iloc[start:end,:]
    data_text = shortened_dataset.to_string(index=False)
    prompt = instructions + data_text
    print('Calling ChatGPT API, wait up!')

    try:
        response = call_chatgpt(prompt)
        tmp_df = parse_response_into_pf(response)
        tmp_df = pd.concat([shortened_dataset.reset_index(), tmp_df.reset_index()], axis=1)
        full_2_df = pd.concat([full_2_df, tmp_df], ignore_index=True, axis=0)
    except Exception as e:
        print(f"Failed to process data for indices {start} to {end}: {e}")

Processing rows from index 20 to 30
Calling ChatGPT API, wait up!
Processing rows from index 30 to 40
Calling ChatGPT API, wait up!


In [172]:
full_2_df

,index,name,description,to_use,index,Description,Number of Bedrooms,Type of Property,Is any space shared?,Overall vibes/atmosphere
0,21,Charming Room in Amazing East Village Apt. (B),The apartment is among the best and most uniqu...,1.0,1,The ap,1,Apartment,FALSE,ARTSY
1,22,SpaHa Loft: Enormous and Bright,Our luxury loft occupies the entire second flo...,1.0,2,Our lu,4,Loft,FALSE,MODERN
2,23,Lovely EV Artist's Home,I am happy to host responsible visitors as my ...,1.0,3,I am h,1,Apartment,TRUE,COMFORTABLE
3,24,Manhattan Room,Please message me before making booking reques...,1.0,4,Please,1,Room,TRUE,COMFORTABLE
4,25,Beautiful Meatpacking District Loft,The hottest neighborhood in New York City! Now...,1.0,5,The ho,1,Loft,FALSE,CHIC
5,26,Cozy Studio in Great Location!,"This is a great fully furnished studio, lots o...",1.0,6,This is,1,Studio,FALSE,COMFORTABLE
6,28,"Sunny, Spacious Top Floor Haven","A true GEM in Prospect Lefferts Gardens, welco...",1.0,7,A true,2,Apartment,FALSE,MODERN
7,29,2 bedroom Williamsburg Apt - Bedford L stop,I love my apartment and I love being able to o...,1.0,8,I love,2,Apartment,FALSE,HISTORIC
8,30,Charming 1 bed GR8 WBurg LOCATION!,Two blocks from the subway and SO close to the...,1.0,9,Two bl,1,Apartment,FALSE,COSMOPOLITAN
9,31,"Upper Manhattan, New York",A great space in a beautiful neighborhood- min...,1.0,10,A grea,1,Room,FALSE,COMFORTABLE


In [173]:
#gathering datasets 

# indices = [[0, 10], [10,20], [20,30], [30,40]]
indices = [[40, 50], [50, 60], [60, 70], [70, 80]]

full_3_df = pd.DataFrame()

for idx in indices: 
    print(f"Processing rows from index {idx[0]} to {idx[1]}")
    start = idx[0]
    end = idx[1]

    shortened_dataset = df.iloc[start:end,:]
    data_text = shortened_dataset.to_string(index=False)
    prompt = instructions + data_text
    print('Calling ChatGPT API, wait up!')

    try:
        response = call_chatgpt(prompt)
        tmp_df = parse_response_into_pf(response)
        tmp_df = pd.concat([shortened_dataset.reset_index(), tmp_df.reset_index()], axis=1)
        full_3_df = pd.concat([full_3_df, tmp_df], ignore_index=True, axis=0)
    except Exception as e:
        print(f"Failed to process data for indices {start} to {end}: {e}")

Processing rows from index 40 to 50
Calling ChatGPT API, wait up!
Processing rows from index 50 to 60
Calling ChatGPT API, wait up!
Processing rows from index 60 to 70
Calling ChatGPT API, wait up!
Failed to process data for indices 60 to 70: single positional indexer is out-of-bounds
Processing rows from index 70 to 80
Calling ChatGPT API, wait up!
Failed to process data for indices 70 to 80: single positional indexer is out-of-bounds


In [175]:
full_3_df

,index,name,description,to_use,index,Description,Number of Bedrooms,Type of Property,Is any space shared?,Overall vibes/atmosphere
0,43,Entire apartment _Most central location!,Entire apartment will be available while I am ...,1.0,1,Entire,1,Apartment,FALSE,COMFORTABLE
1,44,Luminous Beautiful West Village Studio,"Wonderfully bright, nicely furnished 400-squar...",1.0,2,Luminously,1,Studio,FALSE,CHIC
2,45,Sanctuary in East Flatbush,Come and Stay in a warm and nurturing environm...,1.0,3,Sanctuary in,1,Not Specified,TRUE,ARTSY
3,46,Room with En Suite Bathroom & Deck,"Mature, chilled and quiet",1.0,4,Room with,1,Room,TRUE,COMFORTABLE
4,47,Huge Private Floor at The Waverly,Spacious studio in one of the city’s best neig...,1.0,5,Huge Private,1,Studio,FALSE,COSMOPOLITAN
5,48,Retreat in Williamsburg,"A calm, peaceful, private home away from home.",1.0,6,Retreat in,1,Not Specified,FALSE,COMFORTABLE
6,49,"Sunny Room, Historical Brooklyn Brownstone 30-...",Sunny Room in a lovely Brownstone in Fort Gree...,1.0,7,"Sunny Room,",1,Room,TRUE,HISTORIC
7,50,Lg Rm in Historic Prospect Heights,Cozy space share in the heart of a great neigh...,1.0,8,Lg Rm in,1,Room,TRUE,HISTORIC
8,51,"1 Bedroom & your own Bathroom, Elevator Apartment",Private bedroom with your own bathroom in a 2 ...,1.0,9,1 Bedroom,1,Apartment,FALSE,MODERN
9,52,Luxury Brownstone in Boerum Hill,"Beautiful, large home in great hipster neighbo...",1.0,10,Luxury Browns,1,House,FALSE,CHIC


In [190]:
#gathering datasets 

# indices = [[0, 10], [10,20], [20,30], [30,40]]
indices = [[60, 70], [70, 80]]

full_4_df = pd.DataFrame()

for idx in indices: 
    print(f"Processing rows from index {idx[0]} to {idx[1]}")
    start = idx[0]
    end = idx[1]

    shortened_dataset = df.iloc[start:end,:]
    data_text = shortened_dataset.to_string(index=False)
    prompt = instructions + data_text
    print('Calling ChatGPT API, wait up!')

    try:
        response = call_chatgpt(prompt)
        tmp_df = parse_response_into_pf(response)
        tmp_df = pd.concat([shortened_dataset.reset_index(), tmp_df.reset_index()], axis=1)
        full_4_df = pd.concat([full_4_df, tmp_df], ignore_index=True, axis=0)
    except Exception as e:
        print(f"Failed to process data for indices {start} to {end}: {e}")

Processing rows from index 60 to 70
Calling ChatGPT API, wait up!
Processing rows from index 70 to 80
Calling ChatGPT API, wait up!


In [191]:
full_4_df

,index,name,description,index,Description,Number of Bedrooms,Type of Property,Is any space shared?,Overall vibes/atmosphere
0,69,Large Room pr/h Br/St in Park Slope,"Sleep in a light filled,, quiet room,on a firm...",1,Sleep,1,Apartment,TRUE,COMFORTABLE
1,70,Serene Private Room w own bathroom,Enjoy a great stay in this spacious warm invit...,2,Enjoy,2,Apartment,FALSE,COMFORTABLE
2,72,"Private, sunny two-bedroom near subway & shops",Enjoy peace and quiet and the comforts of home...,3,Enjoy,2,Apartment,TRUE,CHIC
3,74,Clean and convenient 2BR apartment,"Conveniently located 2BR in Ridgewood, NY",4,Conve,2,Apartment,Not Specified,Not Specified
4,75,3 floors of luxury!,Relax and enjoy this newly remodeled 4 bedroom...,5,Relax,4,Townhome,FALSE,MODERN
5,77,S/Rm Br/st PSlope great for Med/Stu,Insect protective cover on extra firm Queen be...,6,Insect,1,Apartment,FALSE,HISTORIC
6,78,City Room - Private Penthouse Apt.,This nicely presented third floor apartment in...,7,This n,1,Apartment,FALSE,COSMOPOLITAN
7,80,Bedroom in my apartment with a private bathroom.,This listing is for a room in my apartment. I ...,8,This l,1,Apartment,FALSE,COMFORTABLE
8,81,Work From Our Home w/ Loft,*Monthly Discount applies automatically<br /><...,9,Month,1,Apartment,TRUE,ARTSY
9,82,Cozy 2 br in sunny Fort Greene apt,I bedroom available in 2 bedroom apt in the fa...,10,I bedr,1,Apartment,FALSE,COMFORTABLE


In [192]:
#gathering datasets 

# indices = [[0, 10], [10,20], [20,30], [30,40]]
indices = [[70, 80], [80, 90], [90, 100], [100, 110], [110, 120]]

full_5_df = pd.DataFrame()

for idx in indices: 
    print(f"Processing rows from index {idx[0]} to {idx[1]}")
    start = idx[0]
    end = idx[1]

    shortened_dataset = df.iloc[start:end,:]
    data_text = shortened_dataset.to_string(index=False)
    prompt = instructions + data_text
    print('Calling ChatGPT API, wait up!')

    try:
        response = call_chatgpt(prompt)
        tmp_df = parse_response_into_pf(response)
        tmp_df = pd.concat([shortened_dataset.reset_index(), tmp_df.reset_index()], axis=1)
        full_5_df = pd.concat([full_5_df, tmp_df], ignore_index=True, axis=0)
    except Exception as e:
        print(f"Failed to process data for indices {start} to {end}: {e}")

Processing rows from index 70 to 80
Calling ChatGPT API, wait up!
Processing rows from index 80 to 90
Calling ChatGPT API, wait up!
Processing rows from index 90 to 100
Calling ChatGPT API, wait up!
Processing rows from index 100 to 110
Calling ChatGPT API, wait up!
Processing rows from index 110 to 120
Calling ChatGPT API, wait up!


In [255]:
start_index = 420
index_ranges = [[i, i+10] for i in range(start_index, start_index + 100, 10)]
print(index_ranges)

[[420, 430], [430, 440], [440, 450], [450, 460], [460, 470], [470, 480], [480, 490], [490, 500], [500, 510], [510, 520]]


In [195]:
#gathering datasets 

# indices = [[0, 10], [10,20], [20,30], [30,40]]
indices = [[120, 130], [130, 140], [140, 150], [150, 160], [160, 170], [170, 180], [180, 190], [190, 200], [200, 210], [210, 220]]

full_6_df = pd.DataFrame()

for idx in indices: 
    print(f"Processing rows from index {idx[0]} to {idx[1]}")
    start = idx[0]
    end = idx[1]

    shortened_dataset = df.iloc[start:end,:]
    data_text = shortened_dataset.to_string(index=False)
    prompt = instructions + data_text
    print('Calling ChatGPT API, wait up!')

    try:
        response = call_chatgpt(prompt)
        tmp_df = parse_response_into_pf(response)
        tmp_df = pd.concat([shortened_dataset.reset_index(), tmp_df.reset_index()], axis=1)
        full_6_df = pd.concat([full_6_df, tmp_df], ignore_index=True, axis=0)
    except Exception as e:
        print(f"Failed to process data for indices {start} to {end}: {e}")

Processing rows from index 120 to 130
Calling ChatGPT API, wait up!
Processing rows from index 130 to 140
Calling ChatGPT API, wait up!
Processing rows from index 140 to 150
Calling ChatGPT API, wait up!
Processing rows from index 150 to 160
Calling ChatGPT API, wait up!
Processing rows from index 160 to 170
Calling ChatGPT API, wait up!
Processing rows from index 170 to 180
Calling ChatGPT API, wait up!
Processing rows from index 180 to 190
Calling ChatGPT API, wait up!
Processing rows from index 190 to 200
Calling ChatGPT API, wait up!
Processing rows from index 200 to 210
Calling ChatGPT API, wait up!
Processing rows from index 210 to 220
Calling ChatGPT API, wait up!


In [198]:
indices = [[220, 230], [230, 240], [240, 250], [250, 260], [260, 270], [270, 280], [280, 290], [290, 300], [300, 310], [310, 320]]

full_7_df = pd.DataFrame()

for idx in indices: 
    print(f"Processing rows from index {idx[0]} to {idx[1]}")
    start = idx[0]
    end = idx[1]

    shortened_dataset = df.iloc[start:end,:]
    data_text = shortened_dataset.to_string(index=False)
    prompt = instructions + data_text
    print('Calling ChatGPT API, wait up!')

    try:
        response = call_chatgpt(prompt)
        tmp_df = parse_response_into_pf(response)
        tmp_df = pd.concat([shortened_dataset.reset_index(), tmp_df.reset_index()], axis=1)
        full_7_df = pd.concat([full_7_df, tmp_df], ignore_index=True, axis=0)
    except Exception as e:
        print(f"Failed to process data for indices {start} to {end}: {e}")

Processing rows from index 220 to 230
Calling ChatGPT API, wait up!
Processing rows from index 230 to 240
Calling ChatGPT API, wait up!
Processing rows from index 240 to 250
Calling ChatGPT API, wait up!
Processing rows from index 250 to 260
Calling ChatGPT API, wait up!
Processing rows from index 260 to 270
Calling ChatGPT API, wait up!
Processing rows from index 270 to 280
Calling ChatGPT API, wait up!
Processing rows from index 280 to 290
Calling ChatGPT API, wait up!
Processing rows from index 290 to 300
Calling ChatGPT API, wait up!
Processing rows from index 300 to 310
Calling ChatGPT API, wait up!
Processing rows from index 310 to 320
Calling ChatGPT API, wait up!


In [204]:
full_7_df.shape

(100, 9)

In [245]:
final_df = pd.concat([full_df, full_2_df, full_3_df, full_4_df, full_5_df, full_6_df, full_7_df], 
                      ignore_index=True, 
                      axis=0)

In [248]:
final_df.to_clipboard(index=False)

In [250]:
indices = [[320, 330], [330, 340], [340, 350], [350, 360], [360, 370], [370, 380], [380, 390], [390, 400], [400, 410], [410, 420]]

full_8_df = pd.DataFrame()

for idx in indices: 
    print(f"Processing rows from index {idx[0]} to {idx[1]}")
    start = idx[0]
    end = idx[1]

    shortened_dataset = df.iloc[start:end,:]
    data_text = shortened_dataset.to_string(index=False)
    prompt = instructions + data_text
    print('Calling ChatGPT API, wait up!')

    try:
        response = call_chatgpt(prompt)
        tmp_df = parse_response_into_pf(response)
        tmp_df = pd.concat([shortened_dataset.reset_index(), tmp_df.reset_index()], axis=1)
        full_8_df = pd.concat([full_8_df, tmp_df], ignore_index=True, axis=0)
    except Exception as e:
        print(f"Failed to process data for indices {start} to {end}: {e}")

Processing rows from index 320 to 330
Calling ChatGPT API, wait up!
Processing rows from index 330 to 340
Calling ChatGPT API, wait up!
Processing rows from index 340 to 350
Calling ChatGPT API, wait up!
Processing rows from index 350 to 360
Calling ChatGPT API, wait up!
Processing rows from index 360 to 370
Calling ChatGPT API, wait up!
Processing rows from index 370 to 380
Calling ChatGPT API, wait up!
Processing rows from index 380 to 390
Calling ChatGPT API, wait up!
Processing rows from index 390 to 400
Calling ChatGPT API, wait up!
Processing rows from index 400 to 410
Calling ChatGPT API, wait up!
Processing rows from index 410 to 420
Calling ChatGPT API, wait up!


In [251]:
full_8_df.to_clipboard(index=False)

In [256]:
indices = [[420, 430], [430, 440], [440, 450], [450, 460], [460, 470], [470, 480], [480, 490], [490, 500], [500, 510], [510, 520]]

full_9_df = pd.DataFrame()

for idx in indices: 
    print(f"Processing rows from index {idx[0]} to {idx[1]}")
    start = idx[0]
    end = idx[1]

    shortened_dataset = df.iloc[start:end,:]
    data_text = shortened_dataset.to_string(index=False)
    prompt = instructions + data_text
    print('Calling ChatGPT API, wait up!')

    try:
        response = call_chatgpt(prompt)
        tmp_df = parse_response_into_pf(response)
        tmp_df = pd.concat([shortened_dataset.reset_index(), tmp_df.reset_index()], axis=1)
        full_9_df = pd.concat([full_9_df, tmp_df], ignore_index=True, axis=0)
    except Exception as e:
        print(f"Failed to process data for indices {start} to {end}: {e}")


Processing rows from index 420 to 430
Calling ChatGPT API, wait up!
Processing rows from index 430 to 440
Calling ChatGPT API, wait up!
Processing rows from index 440 to 450
Calling ChatGPT API, wait up!
Processing rows from index 450 to 460
Calling ChatGPT API, wait up!
Processing rows from index 460 to 470
Calling ChatGPT API, wait up!
Processing rows from index 470 to 480
Calling ChatGPT API, wait up!
Processing rows from index 480 to 490
Calling ChatGPT API, wait up!
Processing rows from index 490 to 500
Calling ChatGPT API, wait up!
Processing rows from index 500 to 510
Calling ChatGPT API, wait up!
Processing rows from index 510 to 520
Calling ChatGPT API, wait up!


In [257]:
full_9_df.to_clipboard(index=False)

In [262]:
# instructions = """please generate the table again,
# this time I'm giving you only the description, 
# please create a table with these columns: number of bedrooms, number of bathrooms, neighborhood, 
# shared (indicates if anything in the accommodation will be shared): """
instructions = """

I have a list of Airbnb property descriptions, 
and I need to extract and compile key information 
into a structured format suitable for analysis. 
When the information is not present, it can be indicated "Not Specified". 
Could you provide (using your understanding and not regular expressions) 
the following attributes from each description 

0. Description (include the first 5 characters of the original description)
1. Number of Bedrooms
2. Type of Property (e.g., studio, apartment, loft, etc.)
3. Is any space shared? (return TRUE OR FALSE)
4. Overall vibes/atmosphere (return one of the following labels, see detail explanation below: MODERN, CHIC, ARTSY, HISTORIC, COMFORTABLE, PLAIN)

CONTENT INSTRUCTIONS:

- Instructions for attribute 1. Number of Bedrooms: 
When you can infer there's only 1 bed, for e.g., studio or loft, return 1.
Keep the output numerical only, e.g. 1

- Instructions for attribute 3. Is any space shared?:
If there's any indication the space is shared, e.g. explicit mention, or wording referring to the space as a "room"

- Instructions for attribute 4. Overall vibes/atmosphere: 
Modern: descriptions that feature contemporary design, sleek furnishings, and often incorporate new appliances.
Chic: Label as chic where the style is sophisticated, fashionable, and stylish, typically featuring trendy decor and a cosmopolitan vibe.
Artsy: Identify as artsy if the home includes unique artistic elements, vibrant colors, or displays of local artwork, creating a creative and inspirational environment.
Historic: Classify as historic if the property retains architectural features from a specific era or is located in a notable NYC neighborhood with rich historical significance.
Comfortable: If features point to coziness, comfort.
Plain: minimalist apartment with just the basics.

- Instructions for all columns: 
Please return the exact text in the way that it appears in the original description.

OUTPUT FORMATTING: table format, with header (the column attributes), followed by each row
with the information obtained.
To making parsing easy, let's agree to seperate each row by "\n", 
and 2) seperated with | for each attribute.
DO NOT include any explanation before or after
INCLUDE the original column name in the result

EXAMPLE FORMATTING
"Description|Number of Bedrooms|Type of Property|Special Features|Is any space shared?|Overall vibes/atmosphere"
"Easy access to all downtown|1|Apartment|Fast Wifi, Desk, Shared kitchen|TRUE|ARTSY"

"""

In [260]:
start_index = 0
indices = [[i, i+10] for i in range(start_index, start_index + 200, 10)]

full_10_df = pd.DataFrame()

for idx in indices: 
    print(f"Processing rows from index {idx[0]} to {idx[1]}")
    start = idx[0]
    end = idx[1]

    shortened_dataset = df.iloc[start:end,:]
    data_text = shortened_dataset.to_string(index=False)
    prompt = instructions + data_text
    print('Calling ChatGPT API, wait up!')

    try:
        response = call_chatgpt(prompt)
        tmp_df = parse_response_into_pf(response)
        tmp_df = pd.concat([shortened_dataset.reset_index(), tmp_df.reset_index()], axis=1)
        full_10_df = pd.concat([full_10_df, tmp_df], ignore_index=True, axis=0)
    except Exception as e:
        print(f"Failed to process data for indices {start} to {end}: {e}")


Processing rows from index 0 to 10
Calling ChatGPT API, wait up!
Processing rows from index 10 to 20
Calling ChatGPT API, wait up!
Processing rows from index 20 to 30
Calling ChatGPT API, wait up!
Processing rows from index 30 to 40
Calling ChatGPT API, wait up!
Processing rows from index 40 to 50
Calling ChatGPT API, wait up!
Processing rows from index 50 to 60
Calling ChatGPT API, wait up!
Processing rows from index 60 to 70
Calling ChatGPT API, wait up!
Processing rows from index 70 to 80
Calling ChatGPT API, wait up!
Processing rows from index 80 to 90
Calling ChatGPT API, wait up!
Processing rows from index 90 to 100
Calling ChatGPT API, wait up!
Processing rows from index 100 to 110
Calling ChatGPT API, wait up!
Processing rows from index 110 to 120
Calling ChatGPT API, wait up!
Processing rows from index 120 to 130
Calling ChatGPT API, wait up!
Processing rows from index 130 to 140
Calling ChatGPT API, wait up!
Processing rows from index 140 to 150
Calling ChatGPT API, wait up!


In [261]:
full_10_df.to_clipboard(index=False)

In [263]:
start_index = 200
indices = [[i, i+10] for i in range(start_index, start_index + 200, 10)]

full_11_df = pd.DataFrame()

for idx in indices: 
    print(f"Processing rows from index {idx[0]} to {idx[1]}")
    start = idx[0]
    end = idx[1]

    shortened_dataset = df.iloc[start:end,:]
    data_text = shortened_dataset.to_string(index=False)
    prompt = instructions + data_text
    print('Calling ChatGPT API, wait up!')

    try:
        response = call_chatgpt(prompt)
        tmp_df = parse_response_into_pf(response)
        tmp_df = pd.concat([shortened_dataset.reset_index(), tmp_df.reset_index()], axis=1)
        full_11_df = pd.concat([full_11_df, tmp_df], ignore_index=True, axis=0)
    except Exception as e:
        print(f"Failed to process data for indices {start} to {end}: {e}")


Processing rows from index 200 to 210
Calling ChatGPT API, wait up!
Processing rows from index 210 to 220
Calling ChatGPT API, wait up!
Processing rows from index 220 to 230
Calling ChatGPT API, wait up!
Processing rows from index 230 to 240
Calling ChatGPT API, wait up!
Processing rows from index 240 to 250
Calling ChatGPT API, wait up!
Processing rows from index 250 to 260
Calling ChatGPT API, wait up!
Processing rows from index 260 to 270
Calling ChatGPT API, wait up!
Processing rows from index 270 to 280
Calling ChatGPT API, wait up!
Processing rows from index 280 to 290
Calling ChatGPT API, wait up!
Processing rows from index 290 to 300
Calling ChatGPT API, wait up!
Processing rows from index 300 to 310
Calling ChatGPT API, wait up!
Processing rows from index 310 to 320
Calling ChatGPT API, wait up!
Processing rows from index 320 to 330
Calling ChatGPT API, wait up!
Processing rows from index 330 to 340
Calling ChatGPT API, wait up!
Processing rows from index 340 to 350
Calling Ch

In [266]:
start_index = 400
indices = [[i, i+10] for i in range(start_index, start_index + 200, 10)]

full_12_df = pd.DataFrame()

for idx in indices: 
    print(f"Processing rows from index {idx[0]} to {idx[1]}")
    start = idx[0]
    end = idx[1]

    shortened_dataset = df.iloc[start:end,:]
    data_text = shortened_dataset.to_string(index=False)
    prompt = instructions + data_text
    print('Calling ChatGPT API, wait up!')

    try:
        response = call_chatgpt(prompt)
        tmp_df = parse_response_into_pf(response)
        tmp_df = pd.concat([shortened_dataset.reset_index(), tmp_df.reset_index()], axis=1)
        full_12_df = pd.concat([full_12_df, tmp_df], ignore_index=True, axis=0)
    except Exception as e:
        print(f"Failed to process data for indices {start} to {end}: {e}")


Processing rows from index 400 to 410
Calling ChatGPT API, wait up!
Processing rows from index 410 to 420
Calling ChatGPT API, wait up!
Processing rows from index 420 to 430
Calling ChatGPT API, wait up!
Processing rows from index 430 to 440
Calling ChatGPT API, wait up!
Processing rows from index 440 to 450
Calling ChatGPT API, wait up!
Processing rows from index 450 to 460
Calling ChatGPT API, wait up!
Processing rows from index 460 to 470
Calling ChatGPT API, wait up!
Processing rows from index 470 to 480
Calling ChatGPT API, wait up!
Processing rows from index 480 to 490
Calling ChatGPT API, wait up!
Processing rows from index 490 to 500
Calling ChatGPT API, wait up!
Processing rows from index 500 to 510
Calling ChatGPT API, wait up!
Processing rows from index 510 to 520
Calling ChatGPT API, wait up!
Processing rows from index 520 to 530
Calling ChatGPT API, wait up!
Processing rows from index 530 to 540
Calling ChatGPT API, wait up!
Processing rows from index 540 to 550
Calling Ch

In [267]:
pd.concat([full_10_df, full_11_df, full_12_df]).to_clipboard(index=False)